In [1]:
%env MKL_NUM_THREADS=4
%env MKL_DEBUG_CPU_TYPE=5
%env THEANO_FLAGS=device=cpu, floatX=float32

import pickle
import pandas as pd
import numpy as np
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from collections import Counter
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('matplotlib').setLevel(level=logging.CRITICAL)

plt.rcParams['font.family'] = 'Nimbus Sans'

INPUT_DATA_AUX_STATS = '../data preparation/all_patients_firstDiag_03292021_otherStats.bpkl3'
TIME_DIFF_DATA = '../data preparation/two_dis_time_diff_seqs_03312021.bpkl3'
OUTCOME_DIS = 'Bipolar_Disorder'
# INTERVENTION = 'Abnormal_Spine_Curvature'
TIME_DIFF_UNIT = 100
TIME_DIFF_MIM_THRESHOLD = 100
NUM_CHAINS = 4

HDI_PROB = 0.95

with open(INPUT_DATA_AUX_STATS, 'rb') as f:
    all_aux_stats = pickle.load(f)
    DIS_IDX_DICT = all_aux_stats['dis_to_idx']
    DIS_SYS_DICT = all_aux_stats['dis_to_sys']
    
with open(TIME_DIFF_DATA, 'rb') as f:
    ALL_DIS_PAIR_TIME_DIFF = pickle.load(f)

dis_name_list = []
shift_mean_list = []
shift_hdi_list = []
shift_sig_list = []
alter_waic_list = []
null_waic_list = []
alter_better_list = []
for k, v in DIS_IDX_DICT.items():
    
    INTERVENTION = k
    TIME_DIFF = ALL_DIS_PAIR_TIME_DIFF[(OUTCOME_DIS, 
                                        INTERVENTION, 
                                        DIS_IDX_DICT[OUTCOME_DIS],
                                        DIS_IDX_DICT[INTERVENTION])]
    
    print('\033[94m'+OUTCOME_DIS+' x '+INTERVENTION+'\033[0m')
    
    if (len(TIME_DIFF)) < TIME_DIFF_MIM_THRESHOLD or (INTERVENTION == OUTCOME_DIS):
        continue
        
    dis_name_list.append(INTERVENTION)
    
    data_out_path = f'RDD_experiment_{OUTCOME_DIS}_06202022/{INTERVENTION}'
    
    with open(data_out_path+'/analysis_results_and_plot_data.bpkl3', 'rb') as f:
        rdd_result = pickle.load(f)
        alter_trace = rdd_result['alter.trace']
        alter_waic = rdd_result['alter.waic']
        null_trace = rdd_result['null.trace']
        null_waic = rdd_result['null.waic']
        shift_alter_trace = alter_trace['shift']
    
    shift_mean = shift_alter_trace.mean()
    shift_hdi = az.hdi(shift_alter_trace, hdi_prob=HDI_PROB)
    shift_sig = (shift_hdi[0] * shift_hdi[1] > 0.0)
    
    shift_mean_list.append(shift_mean)
    shift_hdi_list.append(list(shift_hdi))
    shift_sig_list.append(shift_sig)
    
    alter_waic_list.append(alter_waic.waic)
    null_waic_list.append(null_waic.waic)
    alter_better_list.append(alter_waic.waic<null_waic.waic)

env: MKL_NUM_THREADS=4
env: MKL_DEBUG_CPU_TYPE=5
env: THEANO_FLAGS=device=cpu, floatX=float32
Bipolar_Disorder x ADHD
Bipolar_Disorder x Abnormal_Spine_Curvature
Bipolar_Disorder x Acne
Bipolar_Disorder x Acquired_Coagulation_Defect
Bipolar_Disorder x Acquired_Hemolytic_Anemias
Bipolar_Disorder x Acquired_Hypothyroidism
Bipolar_Disorder x Acquired_Limb_Deformities
Bipolar_Disorder x Acquired_Other_Myopathies
Bipolar_Disorder x Acquired_Retinal_Defects
Bipolar_Disorder x Acquired_Visual_Disturbances
Bipolar_Disorder x Acrocephalosyndactyly
Bipolar_Disorder x Acute_Bronchitis
Bipolar_Disorder x Acute_Glomerulonephritis
Bipolar_Disorder x Acute_Renal_Failure
Bipolar_Disorder x Acute_Sinusitis
Bipolar_Disorder x Acute_Upper_Respiratory_Infection
Bipolar_Disorder x Addisons_Disease
Bipolar_Disorder x Adjustment_Disorder
Bipolar_Disorder x Adrenogenital_Disorder
Bipolar_Disorder x Allergic_Rhinitis
Bipolar_Disorder x Alopecia
Bipolar_Disorder x Alpha-1-Antitrypsin_Deficiency
Bipolar_Disorder

In [2]:
rdd_result_df = pd.DataFrame({'Intervention': dis_name_list,
                              'Shift mean': shift_mean_list,
                              'Shift HDI': shift_hdi_list,
                              'Shift HDI significance': shift_sig_list,
                              'Shifted model WAIC': alter_waic_list,
                              'Non-shifted model WAIC': null_waic_list,
                              'Shifted mdodel better?': alter_better_list})

In [3]:
rdd_result_df.to_csv('RDD_experiment_Bipolar_06202022_linearTrend_result.csv')
rdd_result_df.sort_values(by=['Shift mean'])

,Intervention,Shift mean,Shift HDI,Shift HDI significance,Shifted model WAIC,Non-shifted model WAIC,Shifted mdodel better?
19,Neonatal_Disease,-0.314616,"[-4.9456205, 2.8104804]",False,152.021550,146.499935,False
15,General_Ear_Disorder,0.181427,"[-1.8675439, 1.7460246]",False,158.642873,156.278301,False
10,Ear_Infection,0.387146,"[-3.6949615, 4.5418024]",False,250.126900,250.768260,True
8,Atopic_Contact_Dermatitis,0.738948,"[-2.030973, 4.4267454]",False,204.244575,202.418721,False
5,Allergic_Rhinitis,0.763663,"[-1.6836053, 3.299236]",False,206.804138,207.245513,True
16,General_Viral_Infection,0.771778,"[-2.51648, 4.7249246]",False,234.727295,234.504400,False
6,Anxiety_Phobic_Disorder,0.784907,"[-1.3755461, 3.461139]",False,136.887574,137.512827,True
2,Acute_Sinusitis,0.810992,"[-2.1425915, 3.5909832]",False,200.987286,202.363691,True
3,Acute_Upper_Respiratory_Infection,0.820617,"[-4.4113894, 7.3524923]",False,273.483946,266.590242,False
13,Functional_Digestive_Disorder,1.071691,"[-0.5921366, 2.9945438]",False,166.107352,167.254964,True


In [4]:
rdd_result_df_sig = rdd_result_df.loc[rdd_result_df['Shift HDI significance']==True]
rdd_result_df_sig = rdd_result_df_sig.sort_values(by=['Shift mean'])
rdd_result_df_sig.reset_index(inplace=True)
rdd_result_df_sig.to_csv('RDD_experiment_Bipolar_06202022_linearTrend_sig_result.csv')
rdd_result_df_sig

,index,Intervention,Shift mean,Shift HDI,Shift HDI significance,Shifted model WAIC,Non-shifted model WAIC,Shifted mdodel better?
0,14,Fungal_Infection,2.523248,"[0.86025757, 4.3656406]",True,141.073183,150.061100,True
1,12,Eye_Infection,2.613722,"[0.12513217, 4.7196045]",True,212.672439,220.564909,True
2,0,ADHD,8.354613,"[0.8755515, 17.19573]",True,174.044605,177.765982,True
